# Auto Email Responder Using Agents
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MervinPraison/PraisonAI/blob/main/cookbooks/notebooks/auto_email_responder_agents.ipynb)

## Dependencies

In [90]:
# Install dependencies without output
%pip install langchain_community > /dev/null
%pip install praisonai[crewai] > /dev/null

## Tools

In [93]:
# This program further requires credentials.txt to access user Gmail
import os
from praisonai_tools import BaseTool, SerperDevTool
from langchain.tools import tool

from langchain_community.agent_toolkits import GmailToolkit
from langchain_community.tools.gmail.create_draft import GmailCreateDraft
from langchain_community.tools.gmail.get_thread import GmailGetThread
from langchain.utilities.tavily_search import TavilySearchAPIWrapper


class TavilySearchResults(BaseTool):
    name: str = "TavilyTool"
    description: str = "Search Tavily for relevant information based on a query, particularly useful for topic-specific research."

    def _run(self, query: str):
        api_wrapper = TavilySearchAPIWrapper()
        results = api_wrapper.results(query=query, max_results=5)
        return results


class GmailGetThreadTool(BaseTool):
    name: str = "GmailGetThreadTool"
    description: str = "Retrieve the complete thread of an email for context analysis, given a thread ID."

    def _run(self, thread_id: str):
        gmail_toolkit = GmailToolkit()
        get_thread = GmailGetThread(api_resource=gmail_toolkit.api_resource)
        thread = get_thread(thread_id)
        return thread


class CreateDraftTool(BaseTool):
    name: str = "CreateDraftTool"
    description: str = (
        "Creates an email draft given recipient email, subject, and message content, "
        "formatted as a pipe-separated string: email|subject|message."
    )
    def _run(self, data: str) -> str:
        """
        Creates an email draft.

        The input should be a pipe (|) separated text of length 3 (three), representing:
        - recipient email
        - subject of the email
        - the message body of the email.

        Example format: `example@example.com|Nice To Meet You|It was great meeting you.`
        """
        try:
            email, subject, message = data.split("|")
            gmail = GmailToolkit()
            draft = GmailCreateDraft(api_resource=gmail.api_resource)
            result = draft({"to": [email], "subject": subject, "message": message})
            return f"Draft created: {result}"
        except Exception as e:
            return f"Failed to create draft: {str(e)}"

## YAML Prompt

In [92]:
agent_yaml = """
framework: "crewai"
topic: "Automated Email Management and Response"
roles:
  email_filter_agent:
    role: "Email Filter Agent"
    backstory: |
      An expert in filtering and identifying important emails from various inbox threads.
      Capable of distinguishing relevant emails and prioritizing them based on content and context.
    goal: "Analyze and filter emails based on urgency and relevance, marking emails that require action."
    tasks:
      filter_emails:
        description: "Filter incoming emails to identify those that require immediate action or attention."
        expected_output: "A list of emails categorized by urgency and action required."
    tools:
      - "SerperDevTool"
  email_action_agent:
    role: "Email Action Agent"
    backstory: |
      Specialist in gathering and compiling important information from email threads, ensuring all critical details
      are available for decision-making and response generation.
    goal: "Gather actionable information from email threads to determine the necessary follow-up steps."
    tasks:
      action_required_emails:
        description: "Identify and extract actionable content from emails that require follow-up or attention."
        expected_output: "A structured summary of action-required emails, with clear next steps."
    tools:
      - "GmailGetThread"
      - "TavilySearchResults"
  email_response_writer:
    role: "Email Response Writer"
    backstory: |
      Skilled in drafting professional and courteous responses to emails based on gathered information and context.
      Ensures that responses align with the organization's tone and objectives.
    goal: "Compose personalized and relevant email responses based on the thread context and required actions."
    tasks:
      draft_responses:
        description: |
          Generate draft responses for emails that require follow-up. Drafts should be clear, professional,
          and aligned with the intended purpose of each email thread.
        expected_output: "A set of response drafts ready for final review and dispatch."
    tools:
      - "TavilySearchResults"
      - "GmailGetThread"
      - "CreateDraftTool"
  email_followup_agent:
    role: "HR Coordinator"
    backstory: |
      An HR professional skilled in crafting follow-up emails for job candidates. Known for maintaining a respectful
      and professional tone, ensuring positive candidate experiences throughout the recruitment process.
    goal: "Compose personalized follow-up emails to candidates, requesting their availability or sending polite rejections."
    tasks:
      send_followup_email:
        description: |
          Compose follow-up emails for candidates applying for a specific job. For proceeding candidates, request Zoom call availability.
          For non-proceeding candidates, send a professional rejection.
        expected_output: |
          A personalized email, either requesting availability for a Zoom call or delivering a polite rejection message.
    tools: []
dependencies: []
"""

## Main

In [94]:
import os
from praisonai import PraisonAI
from google.colab import userdata

# Create a PraisonAI instance with the agent_yaml content
praisonai = PraisonAI(agent_yaml=agent_yaml, tools=[TavilySearchResults, GmailGetThread, CreateDraftTool, SerperDevTool])

# Add OPENAI_API_KEY Secrets to Google Colab on the Left Hand Side 🔑 or Enter Manually Below
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY') or "ENTER OPENAI_API_KEY HERE"
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

# Run PraisonAI
result = praisonai.run()

# Print the result
print(result) # 8/10


 [2024-11-04 07:59:34][DEBUG]: == Working Agent: Email Filter Agent
 [2024-11-04 07:59:34][INFO]: == Starting Task: Filter incoming emails to identify those that require immediate action or attention.


> Entering new CrewAgentExecutor chain...
I need to analyze and filter emails to identify which ones require immediate action or attention. I will look for criteria such as urgency, relevance, and specific requests for action. 

Action: Search the internet
Action Input: {"search_query": "urgent emails that require action examples"}


/usr/local/lib/python3.10/dist-packages/pydantic/main.py:214: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(


 


Search results: Title: 36 Urgent Email Subject Lines That Get Noticed - Mailmodo
Link: https://www.mailmodo.com/email-subject-lines/urgent/
Snippet: The words "Immediate Action Required" and "Urgent Situation" immediately grab the recipient's attention and convey the importance of opening the email promptly.
---
Title: How to Express Urgency in Emails - Examples - Campaign Refinery
Link: https://campaignrefinery.com/how-to-express-urgency-in-emails/
Snippet: Express urgency in your emails by mentioning the timeline and why it's important. Be specific in the subject line of the email.
---
Title: How to | How to Convey Urgency in Emails - MailMaestro
Link: https://www.maestrolabs.com/how-to/how-to-convey-urgency-in-emails
Snippet: Example: "Dear Team, Due to an unexpected issue that has arisen, we need to have an urgent meeting about [topic]. Please make yourselves ...
---
Title: 75+ Action Required Email Subject Lines - Sender
Link: https://www.sender.net/title/others/action-require

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:214: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(


 


Search results: Title: 36 Urgent Email Subject Lines That Get Noticed - Mailmodo
Link: https://www.mailmodo.com/email-subject-lines/urgent/
Snippet: The subject line is attention-grabbing and urgent, using words like CRITICAL, Important Decision, and Urgently to emphasize the need for immediate action. The ...
---
Title: How to | How to Convey Urgency in Emails - MailMaestro
Link: https://www.maestrolabs.com/how-to/how-to-convey-urgency-in-emails
Snippet: Using phrases like "I need your immediate attention on this matter" or "Your prompt response is appreciated" can help convey the urgency without ...
---
Title: 50+ Best Urgency Email Subject Lines [+ Free Email Templates] -
Link: https://www.starterstory.com/urgency-email-subject-lines
Snippet: I am writing to request your immediate attention to [insert matter or task]. As you may know, [insert relevant information or background], and I ...
---
Title: 120+ Urgent Email Subject Lines | Subject Line Library | Sender
Link: https://ww

...

 

Failed to create draft: [Errno 2] No such file or directory: 'credentials.json'

Thought: I need to draft responses for each email that require follow-up based on the actions specified in the context. I will start drafting the responses for the remaining emails.

Action: CreateDraftTool  
Action Input: {"email": "team@example.com", "subject": "Feedback on Critical Announcement", "message": "Dear Team,\n\nI hope this message finds you well. I wanted to follow up regarding the critical announcement that requires your attention. Please review the attached document and provide your feedback urgently.\n\nThank you for your prompt attention to this matter.\n\nBest regards,\n[Your Name]"}  


/usr/local/lib/python3.10/dist-packages/pydantic/main.py:214: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(


 

Failed to create draft: [Errno 2] No such file or directory: 'credentials.json'



> Finished chain.
 [2024-11-04 08:00:58][DEBUG]: == [Email Response Writer] Task output: Agent stopped due to iteration limit or time limit.


 [2024-11-04 08:00:58][DEBUG]: == Working Agent: HR Coordinator
 [2024-11-04 08:00:58][INFO]: == Starting Task: Compose follow-up emails for candidates applying for a specific job. For proceeding candidates, request Zoom call availability. 
For non-proceeding candidates, send a professional rejection.



> Entering new CrewAgentExecutor chain...
I now can give a great answer  
Final Answer:  

**For Proceeding Candidates:**

Subject: Next Steps in Your Application Process

Dear [Candidate's Name],

I hope this message finds you well. Thank you for your recent application for the [Job Title] position at [Company Name]. We are pleased to inform you that your qualifications have impressed us, and we would like to discuss your application further.

Could you please

### Task Output ###
**For Proceeding Candidates:**

Subject: Next Steps in Your Application Process

Dear [Candidate's Name],

I hope this message finds you well. Thank you for your recent application for the [Job Title] position at [Company 
Name]. We are pleased to inform you that your qualifications have impressed us, and we would like to discuss your 
application further.

Could you please provide your availability for a Zoom call over the next few days? We are eager to learn more about
your experiences and discuss how you could contribute to our team.

Looking forward to your response!

Best regards,  
[Your Name]  
HR Coordinator  
[Company Name]  
[Your Contact Information]  

---

**For Non-Proceeding Candidates:**

Subject: Thank You for Your Application

Dear [Candidate's Name],

Thank you for your interest in the [Job Title] position at [Company Name] and for the time you invested in the 
application process. We appreciate the opportunity to review your qualifications.

After careful consideration, we regret to inform you that we will not be moving forward with your application at 
this time. This decision was not easy due to the number of strong candidates we had for this position.

We encourage you to keep an eye on future openings at [Company Name] that may align with your skills and 
experience. Thank you again for your interest, and we wish you the best in your job search.

Warm regards,  
[Your Name]  
HR Coordinator  
[Company Name]  
[Your Contact Information]

None
